# Library import

In [1]:
# !pip install albumentations==0.4.6


In [2]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

import wandb

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jaeook (use `wandb login --relogin` to force relogin)


True

# Dataset 생성

In [4]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)

        return image
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

# inference function

In [5]:
def inference_fn(test_data_loader, model, device, score):
    outputs = []
    
    #####----------------------------------------------------------------####
    # this is the order in which my classes will be displayed
    display_ids = {"General trash" : 0, "Paper" : 1, "Paper pack" : 2, "Metal" : 3, "Glass" : 4,
               "Plastic" : 5, "Styrofoam": 6, "Plastic bag" : 7, "Battery" : 8, "Clothing": 9}
    # this is a revese map of the integer class id to the string class label
    class_id_to_label = { int(v) : k for k, v in display_ids.items()}
    
    columns = ["images"]
    test_table = wandb.Table(columns=columns)  # wandb table 생성
    #####----------------------------------------------------------------####

    for images in tqdm(test_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = list(image.to(device) for image in images)
        output = model(images)

        for img,out in zip(images,output):
            all_boxes=[]
            outputs.append({'boxes': out['boxes'].tolist(), 'scores': out['scores'].tolist(), 'labels': out['labels'].tolist()})
    
    #####-------------------------wandb log에 bbox 나타내기 위한 코드------------------------------------------####
            for b,s,l in zip(out['boxes'].tolist(), out['scores'].tolist(), out['labels'].tolist()):
                if s > score:
                    box_data = {"position": {
                        "minX": b[0],
                        "maxX": b[2],
                        "minY": b[1],
                        "maxY": b[3] },
                        "class_id" : l-1,
                        "box_caption" : "%s (%.3f)" % (class_id_to_label[l-1], s),
                        "domain" : "pixel",
                        "scores" : { "score" : s }}
                    all_boxes.append(box_data)

            test_table.add_data(wandb.Image(img, boxes={"predictions":{"box_data": all_boxes, "class_labels": class_id_to_label}}))
            # 만들어 둔 wandb table에 wandb.Image 함수를 사용해서  이미지와 bbox coordination을 추가해줍니다.
    
    wandb.run.log({"table": test_table})  # test_table을 wandb log에 올립니다.
    #####-----------------------------------------------------------------------####
    return outputs

# Main

In [6]:
score_threshold = 0.05

In [7]:
def main():
    name = "kfold_augmentation"
    #####----------------------------------------------------------------####
    wandb.init(project="object_detection", entity="cv-3-bitcoin", reinit=True)
    wandb.run.name = "jaeook_test_"+name+"_inference" # 실험의 이름
    #####----------------------------------------------------------------####
    
    annotation = '../../dataset/test.json' # annotation 경로
    data_dir = '../../dataset' # dataset 경로
    test_dataset = CustomDataset(annotation, data_dir)
    check_point = './checkpoints/faster_rcnn_torchvision_checkpoints_'+ name + '.pth' # 체크포인트 경로
    

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=4
    )
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)
    
    # torchvision model 불러오기
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 11  # 10 class + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.to(device)
    model.load_state_dict(torch.load(check_point))
    model.eval()
    
    outputs = inference_fn(test_data_loader, model, device, score_threshold)
    prediction_strings = []
    file_names = []
    coco = COCO(annotation)

    # submission 파일 생성
    for i, output in enumerate(outputs):
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
            if score > score_threshold: 
                # label[1~10] -> label[0~9]
                prediction_string += str(label-1) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                    box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv('./faster_rcnn_torchvision_submission_'+name+'.csv', index=None)
    print(submission.head())

In [8]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
cuda


100%|██████████| 305/305 [33:34<00:00,  6.61s/it]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9993736147880554 221.09954833984375 53.502...  test/0000.jpg
1  5 0.8953014016151428 139.22470092773438 0.0 49...  test/0001.jpg
2  1 0.9755319356918335 297.3426513671875 306.014...  test/0002.jpg
3  9 0.9885278940200806 159.986328125 262.8267822...  test/0003.jpg
4  0 0.8055057525634766 191.58035278320312 345.17...  test/0004.jpg
